In [29]:
import os
import random

from sonicrl.worldmodel.autoencoder import autoencoder, load_image

import cv2
import numpy as np
import ipywidgets as widgets

from IPython.display import display

In [51]:
VAE_WEIGHTS = '/home/ben/PycharmProjects/sonicrl/checkpoints_sample/vae.03-35964.729483.hdf5'
ENCODER_WEIGHTS = '/home/ben/PycharmProjects/sonicrl/checkpoints_sample/encoder.03-35964.729483.hdf5'
DECODER_WEIGHTS = '/home/ben/PycharmProjects/sonicrl/checkpoints_sample/<keras.engine.training.Model object at 0x7ffb04a1db38>.03-35964.729483.hdf5'

FRAMES_DIRECTORY = '/home/ben/PycharmProjects/frames/'

In [39]:
def get_image_bytes(path):
    with open(path, 'rb') as f:
        return f.read()

def encode_image(image):
    image = image * 255.
    image = image.astype(np.uint8)
    _, image_bytes = cv2.imencode('.jpg', image)
    return image_bytes.tobytes()

In [50]:
_, encoder, decoder = autoencoder((224, 320, 3))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 320, 3)  0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 224, 320, 3)  84          input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 112, 160, 64) 1792        conv2d_6[0][0]                   
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 112, 160, 64) 36928       conv2d_7[0][0]                   
__________________________________________________________________________________________________
conv2d_9 (

/home/ben/PycharmProjects/sonicrl/sonicrl/worldmodel/autoencoder.py:105: UserWarning: Output "conv2d_10" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "conv2d_10" during training.
  vae.compile(optimizer='rmsprop')


In [52]:
encoder.load_weights(ENCODER_WEIGHTS)
decoder.load_weights(DECODER_WEIGHTS)

In [10]:
paths = [os.path.join(FRAMES_DIRECTORY, fn) for fn in os.listdir(FRAMES_DIRECTORY)]

In [56]:
def autoencode_random_image(_):
    path = random.choice(paths)
    image_bytes = get_image_bytes(path)
    source_widget.value = image_bytes
    image = load_image(path)
    
    z_mean, _, _ = encoder.predict(np.expand_dims(image, 0))
    
    reconstructed = decoder.predict(z_mean)[0]
    reconstructed_bytes = encode_image(reconstructed)
    reconstructed_widget.value = reconstructed_bytes
    
trigger = widgets.Button(description='Random Image')
trigger.on_click(autoencode_random_image)

source_widget = widgets.Image(format='jpg', height=224, width=320)
reconstructed_widget = widgets.Image(format='jpg', height=224, width=320)

image_layout = widgets.HBox([source_widget, reconstructed_widget])
layout = widgets.VBox([trigger, image_layout])
display(layout)
autoencode_random_image(None)